In [146]:
import json
import os
import fnmatch
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [147]:
####### example to access text of json file


s1 = pd.Series([0,'test', 0])
df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
index = 0

for path, dirs, files in os.walk('twitter-english/'):
    if (path.endswith('source-tweet') or path.endswith('replies')):
        for filename in os.listdir(path):
            if filename.endswith('.json'):
                with open(os.path.join(path,filename)) as jsonFile:
                    jsonSourceTweet = json.load(jsonFile)

                    text_tweet = jsonSourceTweet['text']
                    id_tweet = int(re.sub("[^0-9]", "", filename))

                    newline = [id_tweet, text_tweet, index]
                    df.loc[index] = newline

                    index += 1




In [148]:
df.head()

,id,text,label
0,529661510369284096,"@mikeyerxa EXCUSE ME, WHERE'S MY INVITE (to th...",0
1,529661727940431872,@halihamilton @mikeyerxa gonna try to make it ...,1
2,529661317087768576,@mikeyerxa you should send Prince a set of ten...,2
3,529666750330449920,"@NobleRobel Girl, don't be mean. Age ain't n...",3
4,529660296080916480,Prince is playing a secret show in Toronto ton...,4


In [149]:
df_label = pd.read_json('train-key.json')

In [150]:
df_label.columns = ['label', 'b']
df_label.head()

,label,b
501760642928635904,comment,NaN
500270212198174720,comment,NaN
524971210275565568,comment,NaN
552836882770690049,comment,NaN
500289931097296897,support,NaN


In [151]:
for i in range(len(df)):
    try:
        index = df.loc[i].id
        df.loc[i, 'label'] = df_label.loc[str(index)].label
    except KeyError:
        df.loc[i, 'label'] = 'not_found'
df.label.value_counts()

comment      2907
not_found    1049
support       910
query         358
deny          344
Name: label, dtype: int64

In [152]:
'''
#save not found labels because they are the eval data
index_to_save = df[df.label == "not_found"].index
mask_save = df.index.isin(index_to_save)
df_eval = df[mask_save]
df_label_eval = pd.read_json('dev-key.json')
df_label_eval.columns = ['label', 'b']


for i in range(len(df_eval)):
    index = df_eval.iloc[i].id
    df_eval.loc[i, 'label'] = df_label_eval.loc[str(int(index))].label
'''

'\n#save not found labels because they are the eval data\nindex_to_save = df[df.label == "not_found"].index\nmask_save = df.index.isin(index_to_save)\ndf_eval = df[mask_save]\ndf_label_eval = pd.read_json(\'dev-key.json\')\ndf_label_eval.columns = [\'label\', \'b\']\n\n\nfor i in range(len(df_eval)):\n    index = df_eval.iloc[i].id\n    df_eval.loc[i, \'label\'] = df_label_eval.loc[str(int(index))].label\n'

In [153]:

type(df.id)

pandas.core.series.Series

In [154]:
#df_eval.to_csv('eval_data.csv')

In [155]:

#get rid of not found tag

index_to_delete = df[df.label == "not_found"].index
mask = np.logical_not(df.index.isin(index_to_delete))

df_final = df[mask]
df_final.label.value_counts()


comment    2907
support     910
query       358
deny        344
Name: label, dtype: int64

In [156]:
def create_data(path_training, path_answer):
    s1 = pd.Series([0,'test', 0])
    df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
    index = 0

    for path, dirs, files in os.walk(path_training):
        if (path.endswith('source-tweet') or path.endswith('replies')):
            for filename in os.listdir(path):
                if filename.endswith('.json'):
                    with open(os.path.join(path,filename)) as jsonFile:
                        jsonSourceTweet = json.load(jsonFile)

                        text_tweet = jsonSourceTweet['text']
                        id_tweet = int(re.sub("[^0-9]", "", filename))

                        newline = [id_tweet, text_tweet, index]
                        df.loc[index] = newline

                        index += 1

    #rename column into label                    
    df_label = pd.read_json(path_answer)
    first_col = df_label.columns[0]
    df_label.rename(columns={first_col:'label'}, inplace=True)
    
    #add labels
    for i in range(len(df)):
        try:
            index = df.loc[i].id
            df.loc[i, 'label'] = df_label.loc[str(index)].label
        except KeyError:
            df.loc[i, 'label'] = 'not_found'

    #delete unecessary labels
    index_to_delete = df[df.label == "not_found"].index
    mask = np.logical_not(df.index.isin(index_to_delete))
    df_final = df[mask]


    return df_final

#tes_data_csv = create_data('twitter-en-test-data', 'final-eval-key.json')


In [157]:
tweets = df.text

In [158]:
df.loc[9].text

"@davidleyes @EntCity: I'm crying as well... Don't you worry!"

In [159]:
tweets.str.contains('http') & tweets.str.contains('#')

0       False
1       False
2       False
3       False
4       False
        ...  
5563    False
5564    False
5565    False
5566    False
5567     True
Name: text, Length: 5568, dtype: bool

In [160]:
example_tweet = tweets.loc[5567]
example_tweet

"Was Vladimir Putin netralized by an internal coup? Or maybe he's vacationing in Harper's closet:  http://t.co/tlr3cFLEyv #LookingForNarnia"

In [161]:
#remove links
re.sub(r'http\S+', '', example_tweet)          

"Was Vladimir Putin netralized by an internal coup? Or maybe he's vacationing in Harper's closet:   #LookingForNarnia"

In [162]:

import string
def clean_data(text):
    #remove links
    text = re.sub(r'http\S+', '', text)

    #remove punct except @
    punc_to_remove = string.punctuation
    punc_to_remove.replace('@', '')
    text = "".join([char for char in text if char not in punc_to_remove])
          
    #lower case
    return text.lower()


In [163]:
token_tweet = word_tokenize(example_tweet)
token_tweet[1]

'Vladimir'

In [164]:
stop_words = stopwords.words('english')
token_tweet = word_tokenize(example_tweet)
text = [word for word in token_tweet if word not in stop_words]
text


['Was',
 'Vladimir',
 'Putin',
 'netralized',
 'internal',
 'coup',
 '?',
 'Or',
 'maybe',
 "'s",
 'vacationing',
 'Harper',
 "'s",
 'closet',
 ':',
 'http',
 ':',
 '//t.co/tlr3cFLEyv',
 '#',
 'LookingForNarnia']

In [165]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
for i in range(len(df)):
    tweet = df.loc[i].text
    tweet_tokens = word_tokenize(tweet)
    #without stop words
    tweet_tokens_wst = [word for word in tweet_tokens if word not in stop_words]
    tweet_tokens_wst = TreebankWordDetokenizer().detokenize(tweet_tokens_wst)
    df.loc[i, 'text'] = tweet_tokens_wst


In [166]:
from nltk import WordNetLemmatizer
tweet = df.loc[460].text
wn = WordNetLemmatizer()
tokens = word_tokenize(tweet)
text = [wn.lemmatize(word) for word in tokens]
text

['@',
 'DaryusAzad',
 '@',
 'CBCAlerts',
 '@',
 'CBCNews',
 '@',
 'JustinTrudeau',
 ',',
 'radical',
 'Muslim',
 'extremist',
 'making',
 'u',
 'victim',
 '.']

In [167]:
df.label.value_counts()

comment      2907
not_found    1049
support       910
query         358
deny          344
Name: label, dtype: int64

In [168]:

def fill_parent_id(structure):
    while bool(structure) == True:
        source_tweet = list(structure.keys())[0]
        structure = structure.pop(source_tweet)
        for first_child in structure:
            

            df.loc[df.id == int(first_child), "parent_id"] = int(source_tweet)


            try:
                last_child = list(structure.get(first_child))[0]
                df.loc[df.id == int(last_child), "parent_id"] = int(first_child)
            except IndexError:
                continue
            #print("last\n", last_child)
            #print("first\n", first_child)
            



In [169]:
####### example to access text of json file



for path, dirs, files in os.walk('twitter-english/'):

    if path[-1].isdigit():
        if path.endswith('580348081100734464'):

            for _, _, files in os.walk(path):
                for file in files:
                    if file.endswith('structure.json'):


                        with open(os.path.join(path, file)) as jsonFile:

                            structure = json.load(jsonFile)
                            #fill_parent_id(structure)



def fill_parent_id_version_2(structure):

        source_tweet = list(structure.keys())[0]
        structure = structure.pop(source_tweet)
        for first_child in structure:
            
            #df.loc[df.id == int(first_child), "parent_id"] = int(source_tweet)
            global line
            line = structure.get(first_child)

            if (len(line) > 0):
                print(line)
                break
                
             #   break
            #while childs have sons keep going
#            while structure.get(first_child):
                
 #               print(sub_structure.get(first_child))

fill_parent_id_version_2(structure)

{'580350715408859136': [], '580350976026112000': {'580351508971200512': {'580358103255879680': {'580358531850866688': []}, '580418932152102912': []}, '580353327470247936': [], '580354381578522625': [], '580355004294250496': [], '580421457974566912': {'580422091155095552': [], '580422272575533056': []}}, '580351698545319937': [], '580355513050763264': []}


In [170]:
import json
for path, dirs, files in os.walk('twitter-english/'):


    if (path.endswith("replies")):

        for _, _, files in os.walk(path):
            for file in files:

                with open(os.path.join(path,file)) as jsonFile:
                    data = json.load(jsonFile)
                    parent_id = data["in_reply_to_status_id"]
                    tweet_id = re.sub('\D*', '', file)

                    df.loc[df.id == int(tweet_id), "parent_id"] = float(parent_id)

In [171]:
str(test)

'167    @Trial_Watcher1 @CBCAlerts @CBCNews And the ne...\nName: text, dtype: object'

In [184]:
pd.options.display.float_format = '{:.0f}'.format

for i in range(len(df)):

    id_to_search = df.loc[i].parent_id
    
    if (np.isnan(id_to_search) == False):
        try:
            context = df[df.id == id_to_search].text.values[0]

            end_sentence = df.loc[i].text
            df.loc[i, 'text'] = context + ' <REPLY> ' + end_sentence
        except IndexError:
            print('warning')

warning
warning


In [181]:
df.loc[i]

id                                          553495937432432640
text         @ Mickc1303 @ JosephQuilala @ BBCBreaking @ BB...
label                                                not_found
parent_id                                   553490097623269376
Name: 2048, dtype: object

In [185]:
df

,id,text,label,parent_id
0,529661510369284096,Prince playing secret show Toronto tonight . I...,comment,529660296080916480
1,529661727940431872,Prince playing secret show Toronto tonight . I...,comment,529661510369284096
2,529661317087768576,Prince playing secret show Toronto tonight . I...,comment,529660296080916480
3,529666750330449920,Prince playing secret show Toronto tonight . I...,comment,529661317087768576
4,529660296080916480,Prince playing secret show Toronto tonight . I...,support,nan
...,...,...,...,...
5563,577283431089901568,Was Vladimir Putin netralized internal coup? O...,comment,577258317942149120
5564,577282462788677634,Was Vladimir Putin netralized internal coup? O...,comment,577258317942149120
5565,577284294051123201,Was Vladimir Putin netralized internal coup? O...,comment,577258317942149120
5566,577260238316830721,Was Vladimir Putin netralized internal coup? O...,comment,577258317942149120


In [186]:
df.to_csv('ordered_train_data', index =False)